In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from scipy.sparse import csr_matrix


In [2]:

raw_impressions = pd.read_csv(
    "../MIND/train/behaviors.tsv",
    sep='\t',
    header=None,
    names=["impressionId", "userId", "time", "history", "impressions"],
    usecols=["userId", "history", "impressions"]
  )

In [3]:
import re

impressions = raw_impressions

def split_clicked(x):
  m = re.findall(r"(\w+)-1", x)
  if not m:
    raise Exception("didn't find news id")
  return m



def combine_interractions(x):
  history = x["history"].split(" ")
  clicked = re.findall(r"(\w+)-1", x["impressions"])
  return history + clicked


impressions = impressions.dropna()

impressions["newsId"] = impressions.apply(combine_interractions, axis=1)
impressions = impressions.explode("newsId").reset_index()
impressions = impressions.head(10_000)

impressions["click"] = 1

impressions = impressions[["userId", "newsId", "click"]]


/tmp/ipykernel_189431/904933418.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impressions["newsId"] = impressions.apply(combine_interractions, axis=1)


In [ ]:
# Make sparse user-item matrix 

from collab_utils import create_x


X, uid2index, nid2index, index2uid, index2nid = create_x(impressions)

In [6]:
from matrixfactor import MatrixFactorization


model = MatrixFactorization(X)
model.train()

In [21]:
import sys
import os

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add it to sys.path
sys.path.insert(0, parent_dir)

import utils.evaluation  # Import the module
import importlib

importlib.reload(utils.evaluation)  # Reload after modifying the file


utils.evaluation.evaluate_model(model, raw_impressions)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices